# Librerias

In [1]:
#Importar librerias de Scrappy
%pip install scrapy
%pip install spacy
%pip install nltk
#Importar librerias de spacy para español
!python -m spacy download es_core_news_sm
!python -m spacy download es_core_news_md

  Obtaining dependency information for scrapy from https://files.pythonhosted.org/packages/08/66/22ed9609df4b6d94a66512572a11b35943a6cb36dc268f88ebfbede60be1/Scrapy-2.11.0-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.1 MB 435.7 kB/s eta 0:00:08
     - -------------------------------------- 0.1/3.1 MB 880.9 kB/s eta 0:00:04
     --- ------------------------------------ 0.3/3.1 MB 2.2 MB/s eta 0:00:02
     ------ --------------------------------- 0.5/3.1 MB 3.1 MB/s eta 0:00:01
     ------------- -------------------------- 1.0/3.1 MB 4.7 MB/s eta 0:00:01
     ------------------- -------------------- 1.5/3.1 MB 6.0 MB/s eta 0:00:01
     ------------------------- -------------- 2.0/3.1 MB 6.7 MB/s eta 0:00:01
     ------------------------------ --------- 2.4/3.1 MB 7.3 MB/s eta 0:00:01
     -----------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/13/e7/31226298e99a7babec707d2badb1ae0b9f94e262199f57faf08ed899f4ef/spacy-3.7.2-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for spacy-loggers<2.0.0,>=1.0.0 from https://files.pythonhosted.org/packages/33/78/d1a1a026ef3af911159398c939b1509d5c36fe524c7b644f34a5146c4e16/spacy_loggers-1.0.5-py3-none-any.whl.metadata
  Obtaining dependency information for murmurhash<1.1.0,>=0.28.0 from https://files.pythonhosted.org/packages/3b/56/8630be974aeb05868f2058db0ce6f19d85c27adb9b8f733cf69c856afdaa/murmurhash-1.0.10-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for cymem<2.1.0,>=2.0.2 from https://files.pythonhosted.org/packages/35/e0/34b11adc80502f0760ce2892dfdfcd8a7f450acd3147156c98620cb4071d/cymem-2.0.8-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for preshed<3.1.0,>=3.0.2 from https://files.pythonhosted.org/packages/db/e4/d074efb7e8a

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     - -------------------------------------- 0.1/1.5 MB 544.7 kB/s eta 0:00:03
     ----- ---------------------------------- 0.2/1.5 MB 1.6 MB/s eta 0:00:01
     ------------ --------------------------- 0.5/1.5 MB 2.7 MB/s eta 0:00:01
     ------------------------- -------------- 1.0/1.5 MB 4.3 MB/s eta 0:00:01
     ------------------------------------- -- 1.4/1.5 MB 5.3 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 5.3 MB/s eta 0:00:00
  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for regex>=2021.8.3 from https://files.pythonhosted.org/packages/d3/10/6f2d5f8635d7714ad97ce6ade7a643358c4f3e45cde4ed12b7150734a8f3/regex-2023.10.3-cp312-cp

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     --------------------------------------- 0.0/12.9 MB 330.3 kB/s eta 0:00:39
     --------------------------------------- 0.1/12.9 MB 550.5 kB/s eta 0:00:24
     - -------------------------------------- 0.3/12.9 MB 2.3 MB/s eta 0:00:06
     -- ------------------------------------- 0.7/12.9 MB 4.2 MB/s eta 0:00:03
     --- ------------------------------------ 1.2/12.9 MB 5.5 MB/s eta 0:00:03
     ----- ---------------------------------- 1.7/12.9 MB 6.4 MB/s eta 0:00:02
     ------ --------------------------------- 2.2/12.9 MB 7.2 MB/s eta 0:00:02
     -------- ------------------------------- 2.7/12.9 MB 7.7 MB/s eta 0:00:02
     --------- ------------------------------ 3.2/12.9 MB 8.4 MB/s eta 0:00:02
     ---------- ----------------------------- 3.2/12.9 MB 8.0 MB/s eta 0:00:02
     ------------ --------------------------- 4.1/12.9 MB 8.8 MB/s eta 0:00:02
     ------------- -------------------------- 4.4/12.9 MB


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     --------------------------------------- 0.0/42.3 MB 330.3 kB/s eta 0:02:08
     --------------------------------------- 0.1/42.3 MB 525.1 kB/s eta 0:01:21
     --------------------------------------- 0.1/42.3 MB 737.3 kB/s eta 0:00:58
     ---------------------------------------- 0.3/42.3 MB 1.7 MB/s eta 0:00:25
     ---------------------------------------- 0.5/42.3 MB 2.5 MB/s eta 0:00:17
      --------------------------------------- 1.0/42.3 MB 4.0 MB/s eta 0:00:11
     - -------------------------------------- 1.5/42.3 MB 5.1 MB/s eta 0:00:08
     -- ------------------------------------- 2.1/42.3 MB 6.2 MB/s eta 0:00:07
     -- ------------------------------------- 2.7/42.3 MB 7.0 MB/s eta 0:00:06
     --- ------------------------------------ 3.2/42.3 MB 7.3 MB/s eta 0:00:06
     --- ------------------------------------ 3.7/42.3 MB 7.9 MB/s eta 0:00:05
     --- ------------------------------------ 4.2/42.3 M


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#Asegurarse que se tienen todas las librerias incluidas
import pandas as pd
import numpy as np
import spacy
import es_core_news_sm
import es_core_news_md
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

# Funciones

In [3]:
stop_words = set(stopwords.words('english'))
stop_words.add("\"")
stop_words.add("'")
stop_words.add("’")
stop_words.add("”")
stop_words.add("!")
stop_words.add("“")
stop_words.add("?")

def preProcess(path):
    # LEE 
    with open(path, 'r') as f:
        file = f.readlines()
    
    title = file[0][:-1]
    synopsis = " ".join(file[1::])

    #Minusculas
    lower_text = synopsis.lower()

    #Eliminar signos de puntuación
    nlp = es_core_news_sm.load()
    doc = nlp(lower_text)
    tokens=[]
    # revision de signos de puntuacion y en caso de que no sea se agrega
    for token in doc:
        if token.is_punct==False:
            tokens.append(token.text)

    clean_text = " ".join(tokens)

    #Stopwords
    new_tokens = word_tokenize(clean_text)
    clean_tokens = [w for w in new_tokens if not w in stop_words]
    final_text = " ".join(clean_tokens)
    
    # Guarda
    new_path = path[:53] + '-clean' + path[53:]
    with open(new_path, 'w') as f:
        f.write(f'{title}\n{final_text}')
    
    # returns
    return (int(path[54:-4]), clean_tokens)

In [27]:
df = pd.DataFrame()
def vect(df, id, cleantokens):
    current = list(df.columns)
    c = set(cleantokens)
    zvect = [0] * df.shape[0]
    for p in c:
        if p not in current:
            df[p] = zvect
            current.append(p)
    d = dict()
    for it in current:
        if it in c:
            sum = cleantokens.count(it)
            d[it] = sum/len(cleantokens)
        else:
            d[it] = 0
    new_record = pd.DataFrame([d])
    df = pd.concat([df, new_record], ignore_index=True)
    return df

# Web Scrapping

## Crear el proyecto de Scrappy

In [4]:
!scrapy startproject GoodReadsScraper
%cd GoodReadsScraper/GoodReadsScraper/spiders/

Error: scrapy.cfg already exists in C:\Users\Usuario\Documents\University\Recuperacion Web\LibreriaUNAL\Trabajo 2\GoodReadsScraper
c:\Users\Usuario\Documents\University\Recuperacion Web\LibreriaUNAL\Trabajo 2\GoodReadsScraper\GoodReadsScraper\spiders


## Crear un nuevo Spyder

In [5]:
!scrapy genspider GoodReadsSpider https://www.goodreads.com/shelf/show/fiction

C:\Users\Usuario\Documents\University\Recuperacion Web\LibreriaUNAL\Trabajo 2\GoodReadsScraper\GoodReadsScraper\spiders\GoodReadsSpider.py already exists


## Ejecutar el Spyder

In [36]:
!scrapy runspider GoodReadsSpider.py -O ../../ejercicio0.json

2023-11-01 07:43:56 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: GoodReadsScraper)
2023-11-01 07:43:56 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.11.4 (tags/v3.11.4:d2340ef, Jun  7 2023, 05:45:37) [MSC v.1934 64 bit (AMD64)], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.5, Platform Windows-10-10.0.22621-SP0
2023-11-01 07:43:56 [scrapy.addons] INFO: Enabled addons:
[]
2023-11-01 07:43:56 [asyncio] DEBUG: Using selector: SelectSelector
2023-11-01 07:43:56 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2023-11-01 07:43:56 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.windows_events._WindowsSelectorEventLoop
2023-11-01 07:43:56 [scrapy.extensions.telnet] INFO: Telnet Password: 4b557ea832d7945b
2023-11-01 07:43:56 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.ex

## Codigos para depurar la inserción inicial

In [37]:
import os

# Ruta de la carpeta que deseas explorar
carpeta = 'C:\\Users\\Usuario\\Documents\\University\\Recuperacion Web\\LibreriaUNAL\\Trabajo 2\\archivos\\'

# Lista para almacenar los nombres de archivos vacíos
archivos_vacios = []

# Itera a través de los archivos en la carpeta
for nombre_archivo in os.listdir(carpeta):
    ruta_archivo = os.path.join(carpeta, nombre_archivo)
    
    # Verifica si el archivo es un archivo de texto (.txt)
    if os.path.isfile(ruta_archivo) and nombre_archivo.endswith('.txt'):
        # Abre el archivo y verifica si está vacío
        with open(ruta_archivo, 'r') as archivo:
            contenido = archivo.read()
            if not contenido:
                archivos_vacios.append(nombre_archivo)

# Imprime la lista de archivos vacíos
print("Archivos vacíos:")
for archivo in archivos_vacios:
    print(archivo)

Archivos vacíos:
125.txt
1398.txt
1402.txt
1418.txt
1433.txt
269.txt
277.txt
352.txt
441.txt
484.txt
487.txt
515.txt
522.txt
540.txt
551.txt
577.txt
596.txt
61.txt


In [38]:
import os

# Ruta de la carpeta que deseas explorar
carpeta = 'C:\\Users\\Usuario\\Documents\\University\\Recuperacion Web\\LibreriaUNAL\\Trabajo 2\\archivos\\'

# Diccionario para almacenar las primeras líneas de los archivos
primeras_lineas = {}

# Itera a través de los archivos en la carpeta
for nombre_archivo in os.listdir(carpeta):
    ruta_archivo = os.path.join(carpeta, nombre_archivo)
    
    # Verifica si el archivo es un archivo de texto (.txt)
    if os.path.isfile(ruta_archivo) and nombre_archivo.endswith('.txt'):
        # Abre el archivo y obtén su primera línea
        with open(ruta_archivo, 'r') as archivo:
            primera_linea = archivo.readline().strip()
            # Agrega el nombre del archivo al diccionario según su primera línea
            if primera_linea in primeras_lineas:
                primeras_lineas[primera_linea].append(nombre_archivo)
            else:
                primeras_lineas[primera_linea] = [nombre_archivo]

# Filtra el diccionario para obtener las primeras líneas con más de un archivo
primeras_lineas_repetidas = {linea: archivos for linea, archivos in primeras_lineas.items() if len(archivos) > 1}

# Imprime las primeras líneas repetidas y los nombres de los archivos que las comparten
for linea, archivos in primeras_lineas_repetidas.items():
    print(f"Primera línea repetida: {linea}")
    print(f"Archivos que comparten esta primera línea: {archivos}")


Primera línea repetida: Wuthering Heights
Archivos que comparten esta primera línea: ['1.txt', '1051.txt', '349.txt']
Primera línea repetida: The 7 Habits of Highly Effective People: Powerful Lessons in Personal Change
Archivos que comparten esta primera línea: ['1000.txt', '1244.txt', '194.txt']
Primera línea repetida: Thinking, Fast and Slow
Archivos que comparten esta primera línea: ['1001.txt', '1234.txt', '169.txt', '906.txt']
Primera línea repetida: Attached: The New Science of Adult Attachment and How It Can Help You Find—and Keep—Love
Archivos que comparten esta primera línea: ['1003.txt', '1203.txt']
Primera línea repetida: The Highly Sensitive Person: How to Thrive When the World Overwhelms You
Archivos que comparten esta primera línea: ['1005.txt', '1202.txt']
Primera línea repetida: Why We Sleep: Unlocking the Power of Sleep and Dreams
Archivos que comparten esta primera línea: ['1007.txt', '1135.txt']
Primera línea repetida: Just Kids
Archivos que comparten esta primera lí

In [40]:
import os

# Ruta de la carpeta que deseas explorar
carpeta = 'C:\\Users\\Usuario\\Documents\\University\\Recuperacion Web\\LibreriaUNAL\\Trabajo 2\\archivos\\'

# Diccionario para almacenar el contenido de los archivos
contenido_archivos = {}

# Diccionario para almacenar los nombres de archivos duplicados
archivos_duplicados = {}

# Itera a través de los archivos en la carpeta
for nombre_archivo in os.listdir(carpeta):
    ruta_archivo = os.path.join(carpeta, nombre_archivo)
    
    # Verifica si el archivo es un archivo de texto (.txt)
    if os.path.isfile(ruta_archivo) and nombre_archivo.endswith('.txt'):
        # Abre el archivo y obtén su contenido
        with open(ruta_archivo, 'r') as archivo:
            contenido = archivo.read()
            
            # Verifica si el contenido ya existe en el diccionario
            if contenido in contenido_archivos:
                nombre_duplicado = contenido_archivos[contenido]
                if nombre_duplicado not in archivos_duplicados:
                    archivos_duplicados[nombre_duplicado] = [nombre_archivo]
                else:
                    archivos_duplicados[nombre_duplicado].append(nombre_archivo)
            else:
                contenido_archivos[contenido] = nombre_archivo

# Imprime los archivos duplicados
for nombre_archivo, duplicados in archivos_duplicados.items():
    print(f"El archivo {nombre_archivo} es el mismo que: {', '.join(duplicados)}")


El archivo 1.txt es el mismo que: 1051.txt, 349.txt
El archivo 1015.txt es el mismo que: 1102.txt
El archivo 1039.txt es el mismo que: 1125.txt
El archivo 1007.txt es el mismo que: 1135.txt
El archivo 1044.txt es el mismo que: 114.txt, 1231.txt, 656.txt, 760.txt, 922.txt
El archivo 11.txt es el mismo que: 1160.txt, 341.txt, 744.txt
El archivo 1032.txt es el mismo que: 1201.txt, 925.txt
El archivo 1005.txt es el mismo que: 1202.txt
El archivo 1003.txt es el mismo que: 1203.txt
El archivo 1028.txt es el mismo que: 1205.txt, 759.txt
El archivo 1042.txt es el mismo que: 1208.txt, 161.txt, 929.txt
El archivo 1021.txt es el mismo que: 1210.txt
El archivo 1013.txt es el mismo que: 1213.txt
El archivo 1038.txt es el mismo que: 1223.txt, 188.txt, 936.txt
El archivo 1034.txt es el mismo que: 1224.txt
El archivo 1046.txt es el mismo que: 1227.txt
El archivo 1029.txt es el mismo que: 1232.txt
El archivo 1001.txt es el mismo que: 1234.txt, 169.txt, 906.txt
El archivo 1045.txt es el mismo que: 1236.

In [47]:
import os

# Ruta de la carpeta que deseas explorar
carpeta = 'C:\\Users\\Usuario\\Documents\\University\\Recuperacion Web\\LibreriaUNAL\\Trabajo 2\\archivos\\'

# Diccionario para almacenar el contenido de los archivos
contenido_archivos = {}

# Diccionario para almacenar los nombres de archivos duplicados
archivos_duplicados = {}

# Lista para almacenar los nombres de los archivos que se deben borrar
archivos_por_borrar = []

# Itera a través de los archivos en la carpeta
for nombre_archivo in os.listdir(carpeta):
    ruta_archivo = os.path.join(carpeta, nombre_archivo)
    
    # Verifica si el archivo es un archivo de texto (.txt)
    if os.path.isfile(ruta_archivo) and nombre_archivo.endswith('.txt'):
        # Abre el archivo y obtén su contenido
        with open(ruta_archivo, 'r') as archivo:
            contenido = archivo.read()
            
            # Verifica si el contenido ya existe en el diccionario
            if contenido in contenido_archivos:
                nombre_duplicado = contenido_archivos[contenido]
                if nombre_duplicado not in archivos_duplicados:
                    archivos_duplicados[nombre_duplicado] = [nombre_archivo]
                else:
                    archivos_duplicados[nombre_duplicado].append(nombre_archivo)
                
                # Agrega el archivo duplicado a la lista de archivos a borrar
                archivos_por_borrar.append(nombre_archivo)
            else:
                contenido_archivos[contenido] = nombre_archivo

# Guarda los nombres de los archivos a borrar en un archivo de texto
if archivos_por_borrar:
    with open('../archivos_por_borrar.txt', 'w') as archivo_salida:
        for nombre_archivo in archivos_por_borrar:
            archivo_salida.write(nombre_archivo + '\n')
    print("Nombres de archivos a borrar guardados en archivos_por_borrar.txt")
else:
    print("No se deben borrar archivos duplicados.")


Nombres de archivos a borrar guardados en archivos_por_borrar.txt


In [49]:
import os

# Ruta del archivo que contiene los nombres de archivos a borrar
archivo_por_borrar = 'C:\\Users\\Usuario\\Documents\\University\\Recuperacion Web\\LibreriaUNAL\\Trabajo 2\\GoodReadsScraper\\GoodReadsScraper\\archivos_por_borrar.txt'

# Verifica si el archivo existe
if os.path.exists(archivo_por_borrar):
    with open(archivo_por_borrar, 'r') as archivo:
        # Lee el archivo línea por línea
        lineas = archivo.read().splitlines()
        
        # Borra los archivos listados en el archivo
        for nombre_archivo in lineas:
            try:
                os.remove('C:\\Users\\Usuario\\Documents\\University\\Recuperacion Web\\LibreriaUNAL\\Trabajo 2\\archivos\\'+nombre_archivo)
                print(f"Archivo '{nombre_archivo}' ha sido borrado.")
            except FileNotFoundError:
                print(f"El archivo '{nombre_archivo}' no fue encontrado.")
            except Exception as e:
                print(f"Error al borrar '{nombre_archivo}': {str(e)}")
else:
    print(f"El archivo '{archivo_por_borrar}' no existe.")


Archivo '1102.txt' ha sido borrado.
Archivo '1125.txt' ha sido borrado.
Archivo '1135.txt' ha sido borrado.
Archivo '114.txt' ha sido borrado.
Archivo '1160.txt' ha sido borrado.
Archivo '1201.txt' ha sido borrado.
Archivo '1202.txt' ha sido borrado.
Archivo '1203.txt' ha sido borrado.
Archivo '1205.txt' ha sido borrado.
Archivo '1208.txt' ha sido borrado.
Archivo '1210.txt' ha sido borrado.
Archivo '1213.txt' ha sido borrado.
Archivo '1223.txt' ha sido borrado.
Archivo '1224.txt' ha sido borrado.
Archivo '1227.txt' ha sido borrado.
Archivo '1231.txt' ha sido borrado.
Archivo '1232.txt' ha sido borrado.
Archivo '1234.txt' ha sido borrado.
Archivo '1236.txt' ha sido borrado.
Archivo '1237.txt' ha sido borrado.
Archivo '1239.txt' ha sido borrado.
Archivo '1243.txt' ha sido borrado.
Archivo '1244.txt' ha sido borrado.
Archivo '1245.txt' ha sido borrado.
Archivo '1246.txt' ha sido borrado.
Archivo '1249.txt' ha sido borrado.
Archivo '1309.txt' ha sido borrado.
Archivo '1330.txt' ha sido bo

In [50]:
import os
import shutil

# Ruta de la carpeta original con los archivos txt
carpeta_original = 'C:\\Users\\Usuario\\Documents\\University\\Recuperacion Web\\LibreriaUNAL\\Trabajo 2\\archivos\\'

# Ruta de la carpeta donde se copiarán y renombrarán los archivos
carpeta_destino = 'C:\\Users\\Usuario\\Documents\\University\\Recuperacion Web\\LibreriaUNAL\\Trabajo 2\\archivosfinal\\'

# Asegúrate de que la carpeta de destino exista
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)

# Lista los archivos de la carpeta original
archivos_originales = os.listdir(carpeta_original)

# Inicializa un contador para los nombres secuenciales
contador = 1

# Itera a través de los archivos originales
for nombre_archivo in archivos_originales:
    # Verifica si el archivo es un archivo de texto (.txt)
    if nombre_archivo.endswith('.txt'):
        # Construye el nuevo nombre de archivo
        nuevo_nombre = f"{contador}.txt"
        
        # Ruta completa del archivo original
        ruta_original = os.path.join(carpeta_original, nombre_archivo)
        
        # Ruta completa del archivo de destino
        ruta_destino = os.path.join(carpeta_destino, nuevo_nombre)
        
        # Copia y renombra el archivo a la carpeta de destino
        shutil.copy(ruta_original, ruta_destino)
        
        print(f"Archivo '{nombre_archivo}' copiado y renombrado como '{nuevo_nombre}' en la carpeta de destino.")
        
        # Incrementa el contador
        contador += 1


Archivo '1.txt' copiado y renombrado como '1.txt' en la carpeta de destino.
Archivo '10.txt' copiado y renombrado como '2.txt' en la carpeta de destino.
Archivo '100.txt' copiado y renombrado como '3.txt' en la carpeta de destino.
Archivo '1000.txt' copiado y renombrado como '4.txt' en la carpeta de destino.
Archivo '1001.txt' copiado y renombrado como '5.txt' en la carpeta de destino.
Archivo '1002.txt' copiado y renombrado como '6.txt' en la carpeta de destino.
Archivo '1003.txt' copiado y renombrado como '7.txt' en la carpeta de destino.
Archivo '1004.txt' copiado y renombrado como '8.txt' en la carpeta de destino.
Archivo '1005.txt' copiado y renombrado como '9.txt' en la carpeta de destino.
Archivo '1006.txt' copiado y renombrado como '10.txt' en la carpeta de destino.
Archivo '1007.txt' copiado y renombrado como '11.txt' en la carpeta de destino.
Archivo '1008.txt' copiado y renombrado como '12.txt' en la carpeta de destino.
Archivo '1009.txt' copiado y renombrado como '13.txt' e

# Pre-procesamiento

## Leer archivo

In [75]:
path = 'C:\\Users\\dgbla\\GitHub\\LibreriaUNAL\\Trabajo 2\\archivos\\4.txt'
with open(path, 'r') as f:
    file = f.readlines()
title = file[0][:-1]
synopsis = " ".join(file[1::])
print(title)
print(synopsis)

To Kill a Mockingbird
The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. "To Kill A Mockingbird" became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.


## Poner todo en minuscula

In [76]:
lower_text = synopsis.lower()
print(lower_text)

the unforgettable novel of a childhood in a sleepy southern town and the crisis of conscience that rocked it. "to kill a mockingbird" became both an instant bestseller and a critical success when it was first published in 1960. it went on to win the pulitzer prize in 1961 and was later made into an academy award-winning film, also a classic.


## Eliminar signos de Puntuacion

In [77]:
nlp = es_core_news_sm.load()
doc = nlp(lower_text)
tokens=[]
# revision de signos de puntuacion y en caso de que no sea se agrega
for token in doc:
  if token.is_punct==False:
    tokens.append(token.text)

clean_text = " ".join(tokens)
print(clean_text)

the unforgettable novel of a childhood in a sleepy southern town and the crisis of conscience that rocked it to kill a mockingbird became both an instant bestseller and a critical success when it was first published in 1960 it went on to win the pulitzer prize in 1961 and was later made into an academy award-winning film also a classic


## Eliminar las Stopwords

In [78]:
stop_words = set(stopwords.words('english'))
stop_words.add("\"")
stop_words.add("'")
stop_words.add("’")
stop_words.add("”")
stop_words.add("!")
stop_words.add("“")
stop_words.add("?")

In [79]:
new_tokens = word_tokenize(clean_text)
print(new_tokens)
clean_tokens = [w for w in new_tokens if not w in stop_words]
print(clean_tokens)
final_text = " ".join(clean_tokens)
print(final_text)

['the', 'unforgettable', 'novel', 'of', 'a', 'childhood', 'in', 'a', 'sleepy', 'southern', 'town', 'and', 'the', 'crisis', 'of', 'conscience', 'that', 'rocked', 'it', 'to', 'kill', 'a', 'mockingbird', 'became', 'both', 'an', 'instant', 'bestseller', 'and', 'a', 'critical', 'success', 'when', 'it', 'was', 'first', 'published', 'in', '1960', 'it', 'went', 'on', 'to', 'win', 'the', 'pulitzer', 'prize', 'in', '1961', 'and', 'was', 'later', 'made', 'into', 'an', 'academy', 'award-winning', 'film', 'also', 'a', 'classic']
['unforgettable', 'novel', 'childhood', 'sleepy', 'southern', 'town', 'crisis', 'conscience', 'rocked', 'kill', 'mockingbird', 'became', 'instant', 'bestseller', 'critical', 'success', 'first', 'published', '1960', 'went', 'win', 'pulitzer', 'prize', '1961', 'later', 'made', 'academy', 'award-winning', 'film', 'also', 'classic']
unforgettable novel childhood sleepy southern town crisis conscience rocked kill mockingbird became instant bestseller critical success first publi

## Guardar archivo modificado

In [80]:
path = 'C:\\Users\\dgbla\\GitHub\\LibreriaUNAL\\Trabajo 2\\archivos-clean\\3.txt'
with open(path, 'w') as f:
    f.write(f'{title}\n{final_text}')

## Agregar nuevo documento

In [18]:
import os

# Ruta de la carpeta que deseas contar
carpeta = 'C:\\Users\\Usuario\\Documents\\Github\\LibreriaUNAL\\Trabajo 2\\archivos\\'

# Lista todos los archivos en la carpeta
archivos_en_carpeta = os.listdir(carpeta)

# Filtra la lista para contar solo los archivos (no directorios)
archivos = [archivo for archivo in archivos_en_carpeta if os.path.isfile(os.path.join(carpeta, archivo))]

# Cuenta los archivos
cantidad_de_archivos = len(archivos)

def createDocument(title, synopsis):
    
    # Ruta de la carpeta que deseas contar
    carpeta = 'C:\\Users\\Usuario\\Documents\\Github\\LibreriaUNAL\\Trabajo 2\\archivos\\' #Pc trabajo Jairo
    
    # Lista todos los archivos en la carpeta
    archivos_en_carpeta = os.listdir(carpeta)

    # Filtra la lista para contar solo los archivos (no directorios)
    archivos = [archivo for archivo in archivos_en_carpeta if os.path.isfile(os.path.join(carpeta, archivo))]

    # Cuenta los archivos
    cantidad_de_archivos = len(archivos) + 1
    
    path = 'C:\\Users\\Usuario\\Documents\\Github\\LibreriaUNAL\\Trabajo 2\\archivos\\'+ str(cantidad_de_archivos) + '.txt' #Pc trabajo Jairo
        
    with open(path, 'w') as f:
        f.write(f'{title}\n{synopsis}')

In [21]:
import os

title = 'Ingrese el titulo'
sinopsis = 'Ingrese la sinopsis'


# Ruta de la carpeta que contiene los archivos .txt
folder = 'C:\\Users\\Usuario\\Documents\\Github\\LibreriaUNAL\\Trabajo 2\\archivos\\' #Pc del trabajo Jairo

repetedFlag = False

# Recorrer los archivos en la carpeta
for archivo in os.listdir(folder):
    if archivo.endswith('.txt'):
        ruta_archivo = os.path.join(folder, archivo)
        with open(ruta_archivo, 'r') as f:
            lineas = f.readlines()
            if lineas:
                titulo = lineas[0].strip()  # Guarda la primera línea sin espacios en blanco
                if titulo == title: 
                    resto = ''.join(lineas[1:])  # Concatena el resto del contenido
                    if resto == sinopsis:
                        repetedFlag = True
if not repetedFlag:
    createDocument(title, sinopsis)
    print('Sinopsis agregada a la DB')
else:
    print('La sinopsis ya existe')

Sinopsis agregada a la DB


# Vectorizacion

## Dataframe of vectors

In [64]:
df = pd.DataFrame()
df

""


In [81]:
current = list(df.columns)

In [82]:
c = set(clean_tokens)

In [83]:
zvect = [0] * df.shape[0]
for p in c:
    if p not in current:
        df[p] = zvect
        current.append(p)

In [87]:
d = dict()
for it in current:
    if it in c:
        sum = clean_tokens.count(it)
        d[it] = sum/len(clean_tokens)
    else:
        d[it] = 0

In [88]:
df

,capitol,ruins,north,panem,annual,keeps,tv,live,one,hunger,...,classic,kill,rocked,childhood,went,mockingbird,success,later,film,bestseller
0,0.054054,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.054054,0.027027,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258


In [89]:
new_record = pd.DataFrame([d])
df = pd.concat([df, new_record], ignore_index=True)
df

,capitol,ruins,north,panem,annual,keeps,tv,live,one,hunger,...,classic,kill,rocked,childhood,went,mockingbird,success,later,film,bestseller
0,0.054054,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.054054,0.027027,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258


# Base de Datos

In [29]:
for p in range(1,11):
    path = 'C:\\Users\\dgbla\\GitHub\\LibreriaUNAL\\Trabajo 2\\archivos\\' + str(p) + '.txt'
    id, clean = preProcess(path)
    df = vect(df, id, clean)
df

C:\Users\dgbla\AppData\Local\Temp\ipykernel_59292\1710188919.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[p] = zvect
C:\Users\dgbla\AppData\Local\Temp\ipykernel_59292\1710188919.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[p] = zvect
C:\Users\dgbla\AppData\Local\Temp\ipykernel_59292\1710188919.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) inste

C:\Users\dgbla\AppData\Local\Temp\ipykernel_59292\1710188919.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[p] = zvect
C:\Users\dgbla\AppData\Local\Temp\ipykernel_59292\1710188919.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[p] = zvect
C:\Users\dgbla\AppData\Local\Temp\ipykernel_59292\1710188919.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) inste

C:\Users\dgbla\AppData\Local\Temp\ipykernel_59292\1710188919.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[p] = zvect
C:\Users\dgbla\AppData\Local\Temp\ipykernel_59292\1710188919.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[p] = zvect
C:\Users\dgbla\AppData\Local\Temp\ipykernel_59292\1710188919.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) inste

C:\Users\dgbla\AppData\Local\Temp\ipykernel_59292\1710188919.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[p] = zvect
C:\Users\dgbla\AppData\Local\Temp\ipykernel_59292\1710188919.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[p] = zvect
C:\Users\dgbla\AppData\Local\Temp\ipykernel_59292\1710188919.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) inste

,timely,thirty-five,city-wide,hailed,solid,orwell,slums,called,science,abandoned,...,christianity,unravel,reveal,potentially,secret,truth,works,da,evidence,follows
0,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.030303,0.030303,0.030303,0.030303,0.030303,0.030303,0.030303,0.030303,0.030303,0.030303


# Consulta

In [28]:
def prePro(consulta):
    #Minusculas
    lower_text = consulta.lower()

    #Eliminar signos de puntuación
    nlp = es_core_news_sm.load()
    doc = nlp(lower_text)
    tokens=[]
    # revision de signos de puntuacion y en caso de que no sea se agrega
    for token in doc:
        if token.is_punct==False:
            tokens.append(token.text)

    clean_text = " ".join(tokens)

    #Stopwords
    new_tokens = word_tokenize(clean_text)
    clean_tokens = [w for w in new_tokens if not w in stop_words]
    
    # returns
    return clean_tokens

def vecto(df, cleantokens):
    dfa = df.copy()
    current = list(df.columns)
    c = set(cleantokens)
    zvect = [0] * df.shape[0]
    for p in c:
        if p not in current:
            dfa[p] = zvect
            current.append(p)
    d = dict()
    for it in current:
        if it in c:
            sum = cleantokens.count(it)
            d[it] = sum/len(cleantokens)
        else:
            d[it] = 0
    new_record = pd.DataFrame([d])
    return (dfa, new_record)

def relevance(vector, document):
    producto_punto = np.dot(vector, document)
    norma_vector1 = np.linalg.norm(vector)
    norma_vector2 = np.linalg.norm(document)
    coseno_angulo = producto_punto / (norma_vector1 * norma_vector2)
    return coseno_angulo
    
def consulta(df, consulta):
    docs = list()
    tokens = prePro(consulta)
    dfa, vector = vecto(df, tokens)
    vector = vector.to_numpy()
    for index, row in dfa.iterrows():
        r = relevance(vector, row)
        print(r)
        if r>0.5:
            docs.append(index+1)
    return docs

# Ejemplos consultas

In [34]:
docs = consulta(df, "dreams")
docs 

[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]


[]